# Functional Dependency Violations

Example showing how to detect functional dependency violations. Uses the **NYC Parking Violations Issued - Fiscal Year 2014** dataset to identify violations of the functional dependency `Meter Number -> Registration State, Street`.

In [1]:
# Download the full 'DOB Job Application Fiings' dataset.
# Note that the fill is over 300MB in size.

import gzip
import os

from openclean.data.source.socrata import Socrata

datafile = './jt7v-77mi.tsv.gz'

# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        ds = Socrata().dataset('jt7v-77mi')
        print('Downloading ...\n')
        print(ds.name + '\n')
        print(ds.description)
        ds.write(f)


# As an alternative, you can also use the smaller dataset sample that is
# included in the repository.
#
# datafile = './data/jt7v-77mi.tsv.gz'

In [2]:
# Create data stream for the dataset file.

from openclean.pipeline import stream

ds = stream(datafile)

## Data Profiling

In [3]:
# Profile a sample of 1000 rows using the default profiler.

profiles = ds.sample(n=1000, random_state=42).profile()

In [7]:
# Print dataset columns that contain non-empty values together with the most
# frequent data type from the profiling results.

print('Schema\n------')
for col in ds.columns:
    p = profiles.column(col)
    if p['emptyValueCount'] == 1000:
        continue
    print("  '{}' ({})".format(col, p['datatypes'].most_common(1)[0][0]))
    
# Print number of rows.
    
print('\n{} rows.'.format(ds.count()))

Schema
------
  'Summons Number' (int)
  'Plate ID' (str)
  'Registration State' (str)
  'Plate Type' (str)
  'Issue Date' (date)
  'Violation Code' (int)
  'Vehicle Body Type' (str)
  'Vehicle Make' (str)
  'Issuing Agency' (str)
  'Street Code1' (int)
  'Street Code2' (int)
  'Street Code3' (int)
  'Vehicle Expiration Date' (int)
  'Violation Location' (int)
  'Violation Precinct' (int)
  'Issuer Precinct' (int)
  'Issuer Code' (int)
  'Issuer Command' (str)
  'Issuer Squad' (str)
  'Violation Time' (str)
  'Time First Observed' (str)
  'Violation County' (str)
  'Violation In Front Of Or Opposite' (str)
  'Number' (int)
  'Street' (str)
  'Intersecting Street' (str)
  'Date First Observed' (int)
  'Law Section' (int)
  'Sub Division' (str)
  'Violation Legal Code' (str)
  'Days Parking In Effect    ' (str)
  'From Hours In Effect' (str)
  'To Hours In Effect' (str)
  'Vehicle Color' (str)
  'Unregistered Vehicle?' (int)
  'Vehicle Year' (int)
  'Meter Number' (str)
  'Feet From Curb

In [4]:
# Print number of empty cells for each column

profiles.stats()['empty']

Summons Number                          0
Plate ID                                0
Registration State                      0
Plate Type                              0
Issue Date                              0
Violation Code                          0
Vehicle Body Type                       7
Vehicle Make                            9
Issuing Agency                          0
Street Code1                            0
Street Code2                            0
Street Code3                            0
Vehicle Expiration Date                 0
Violation Location                     72
Violation Precinct                      0
Issuer Precinct                         0
Issuer Code                             0
Issuer Command                         69
Issuer Squad                           69
Violation Time                          1
Time First Observed                   896
Violation County                       72
Violation In Front Of Or Opposite      87
Number                            

In [5]:
# Show minimum and maximum value for column 'Meter Number'. We see
# that the column not only contains a lot of emoty values but also
# '-' as an alternative representation for a missing value.

profiles.minmax('Meter Number')

,min,max
str,-,495-0067


## Prepare Data Sample for Demonstration

In [10]:
# Get the first 1000 rows. Convert Street names to upper case. Ignore rows where
# the meter number is undefined (i.e., either an empty string or '-'). Get a
# data frame for the resulting rows.

from openclean.function.eval.domain import IsNotIn

# We only select a subset of columns for this demo.
columns = [
    'Plate ID',
    'Registration State',
    'Plate Type',
    'Meter Number',
    'Street',
    'Vehicle Body Type',
    'Vehicle Make',
    'Vehicle Color'
]

df = ds\
    .select(columns)\
    .where(IsNotIn('Meter Number', set({'-', ''})), limit=1000)\
    .update('Street', str.upper)\
    .to_df()

df.head()

,Plate ID,Registration State,Plate Type,Meter Number,Street,Vehicle Body Type,Vehicle Make,Vehicle Color
661,FXY1858,NY,PAS,407-3018,QUEENS BLVD,SDN,NISSA,GY
780,89988JX,NY,COM,3 -,FRESH POND TRD,VAN,FORD,WHITE
901,FGX2747,NY,PAS,504-3043,,SDN,HONDA,SILVE
2287,23161JR,NY,COM,144-3942,WEST 42 STREET,P-U,FORD,WHITE
2346,47153MC,NY,COM,144-3987,W 40TH ST,SDN,TOYOT,SILV


## Data Cleaning: FD Violations and Value Normalization

In [11]:
# Find violations of the functional dependency Meter Number -> Street.

from openclean.operator.map.violations import fd_violations

groups = fd_violations(df, lhs='Meter Number', rhs='Street')

In [12]:
# List meter numbers that have violations and the number of
# violating values.

for key in groups:
    print('{} {}'.format(key, groups.get(key).shape[0]))

144-3942 4
143-5293 4
144-6383 9
144-3937 9
143-3785 12
144-6376 5
144-6001 3
144-6089 3
143-3791 3
114-9979 2
144-3958 4
144-6453 3
144-3955 8
144-5988 2
144-6088 6
143-3786 2
144-6377 5
143-5983 6
140-5816 3
105-8347 2
140-5756 2
143-3793 2
144-6047 2
140-6386 4
144-3957 2
143-3787 3
144-3959 2
140-5716 2
143-3767 2
140-9944 3
114-9970 2
120-8052 4
431-3003 2
143-5694 4
144-6601 2
301-3609 3
105-8346 2
140-9954 2
145-7412 2
103-4489 2
301-3678 2
143-3901 2
407-2167 2
201-3042 3
407-2206 2
201-3012 2
226-2760 2


In [13]:
# Show street names that cause violations of the functional dependency.

from openclean.operator.collector.count import distinct

print('Meter Number | Street (Count)')
print('=============|===============')
for key in groups:
    conflicts = distinct(groups.get(key), 'Street').most_common()
    street, count = conflicts[0]
    print('{:<12} | {} x {}'.format(key, count, street))
    for street, count in conflicts[1:]:
        print('             | {} x {}'.format(count, street))
    print('-------------|---------------')

Meter Number | Street (Count)
=============|===============
144-3942     | 1 x WEST 42 STREET
             | 1 x WEST 42 ST
             | 1 x WEST 42ND STREET
             | 1 x W 42 STREET
-------------|---------------
143-5293     | 2 x W 45 ST
             | 1 x COLUMBUS AVE
             | 1 x W 45TH ST
-------------|---------------
144-6383     | 8 x 9TH AVE
             | 1 x 9TH AVENUE
-------------|---------------
144-3937     | 5 x WEST 42 STREET
             | 1 x WEST 42 ST
             | 1 x W 42ND ST
             | 1 x WEST 42ND
             | 1 x W 42 ST
-------------|---------------
143-3785     | 3 x WEST 43RD ST
             | 3 x W 43 ST
             | 2 x WEST 43 ST
             | 2 x W 43RD ST
             | 1 x WEST 43RD STREET
             | 1 x W 43 CARTER HOTEL
-------------|---------------
144-6376     | 3 x 8TH AVENUE
             | 2 x 8TH AVE
-------------|---------------
144-6001     | 1 x WEST 38 ST
             | 1 x W 38 ST
             | 1 x W 38TH ST
-

In [10]:
# The Plate ID and Registration State should identify a vehicle uniquely. Here
# we focus on the vehicle color.
#
# Find violations of the FD ['Plate ID', 'Registration State'] -> ['Vehicle Color']
groups = fd_violations(df, lhs=['Plate ID', 'Registration State'], rhs='Vehicle Color')

In [11]:
# List Plate ID and Registration State for vehicles that have multiple colors in the dataset
# together with the set of different colors.

from collections import Counter

for key in groups:
    conflicts = Counter(groups.get(key)['Vehicle Color'].value_counts().keys()).most_common()
    c_format = ', '.join(['{}: {}'.format(val, cnt) for val, cnt in conflicts])
    print('{} = [{}]'.format(key, c_format))

('63272JM', 'NY') = [BWN: 1, BROWN: 1]
('99308MC', 'NY') = [WHITE: 1, WH: 1]
('DNB3070', 'NY') = [BLK: 1, BLACK: 1]
('95743JM', 'NY') = [BL: 1, BLUE: 1]
('63677JM', 'NY') = [BROWN: 1, BRN: 1]
('87071JS', 'NY') = [WHITE: 1, WH: 1]
